In [4]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from datetime import datetime

In [5]:
import tools

In [18]:
data = pd.read_json('data/flink/pr-flat_20220602-13h35m41s_apache_flink_master_prs-brief.txt', lines=True)

In [19]:
data = tools.create_label_dataset(data)
tools.initialize_datetime(data)
data['lifetime'] = tools.add_lifetime_column(data)
data = data.set_index('createdAt')

In [21]:
key_none = '<none>'  # A component name used by the api
key_undefined = '<undefined>'  # introduced by us
data['component'] = key_undefined
data['other'] = key_undefined
data.loc[~data['name'].str.startswith('component='), 'other'] = data.loc[~data['name'].str.startswith('component='), 'name'] 
data.loc[data['name'].str.startswith('component='), 'component'] = data.loc[data['name'].str.startswith('component='), 'name'].str.replace('component=', '') 

In [30]:
data = data.loc[~data['component'].str.contains('<none>') & ~data['component'].str.contains('<undefined>')]

In [42]:
contributions_by_component = data.groupby(['authorLogin', 'component'])['title'].count().reset_index().rename(columns={'title': 'contributions'}).sort_values('contributions', ascending=False).reset_index(drop=True)

In [43]:
with pd.option_context("display.max_rows", 10000000):
    print(contributions_by_component.query('contributions > 10'))

         authorLogin                       component  contributions
0       tillrohrmann            Runtime/Coordination            436
1            twalthr                    TableSQL/API            276
2             zentol                           Tests            266
3             zentol                     BuildSystem            216
4             zentol            Runtime/Coordination            216
5        HuangXingBo                      API/Python            200
6             dianfu                      API/Python            162
7          godfreyhe                TableSQL/Planner            160
8             zentol                 Runtime/Metrics            160
9       lirui-apache                 Connectors/Hive            145
10          zhuzhurk            Runtime/Coordination            124
11        WeiZhong94                      API/Python             96
12             NicoK                 Runtime/Network             95
13      tillrohrmann                           T